In [2]:
# Download libraries for processing XML and HTML with Python

!pip install lxml

import lxml.html as lh

import requests

import pandas as pd

     |████████████████████████████████| 5.5MB 6.4MB/s eta 0:00:01


In [3]:
# Retrieve/download the file from the web-site

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Create a handle, page, to handle the contents of the website
page = requests.get(url)

#Store the contents of the website under doc
doc = lh.fromstring(page.content)

# Check the document on the web-site.
# All the table data ( headers, postal codes) are stored between the HTML tags

#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

#Check the length of each row, it must have 3 columns
#[len(T) for T in tr_elements]

In [4]:
print('\n')
print('# Rows loaded into a list after download :',len(tr_elements))
print('\n')



# Rows loaded into a list after download : 185




In [5]:
#Create empty list  to simply generate the header information
# tr_elements[0]  contains the column headers
# List 'col' will simply be a list of 3 column headers, and an empty list for each header.

col=[]
i=0#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    col.append((name,[]))
print('\n')
print('Header of list: ')
print(col)
print('\n')



Header of list: 
[('Postal Code\n', []), ('Borough\n', []), ('Neighborhood\n', [])]




In [6]:
#Since the first row is the header, data is stored on the second row onwards
# List 'col' is now appended with values for each of the 3 columns

for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
     
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
                
        #If row is not of size 3, the //tr data is not from our table of postal codes
        # This step will filter out any unwanted data such as another table on the web page
    
        if len(T)!=3:
           break
             
        if i==0:
            # If Postal Code is empty, then data is not from the table of postal codes
            if len(data.strip())==0:
                break
        
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        
        #Increment i for the next column
        i+=1

In [7]:
# Checking the final list and verifying rows in each column
print('\n')
print('Checking the final list and verifying rows in each column :')
[len(C) for (title,C) in col]



Checking the final list and verifying rows in each column :


[180, 180, 180]

In [8]:
# Converting the list into a data frame

Dict={title:column for (title,column) in col}

df=pd.DataFrame(Dict)

In [9]:
#The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

df.head()

,Postal Code\n,Borough\n,Neighborhood\n
0,M1A\n,Not assigned\n,\n
1,M2A\n,Not assigned\n,\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [10]:
df.shape

(180, 3)

In [11]:
# Check the tail of the data frame
df.tail()

,Postal Code\n,Borough\n,Neighborhood\n
175,M5Z\n,Not assigned\n,\n
176,M6Z\n,Not assigned\n,\n
177,M7Z\n,Not assigned\n,\n
178,M8Z\n,Etobicoke\n,"Mimico NW, The Queensway West, South of Bloor,..."
179,M9Z\n,Not assigned\n,\n


In [12]:
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

# Count of rows with 'Borough' that is 'Not assigned'

print('\n')
print('Count of rows for each value of Boroughs  :\n', df['Borough\n'].value_counts())
print('\n')
df_Borough_To_drop = df.loc[df['Borough\n'] == 'Not assigned\n']
print('\n')
print('Boroughs to drop due to --Not Assigned-- values :')
print('\n')
print(df_Borough_To_drop)

# Drop rows with ''Borough'' that is ''Not assigned'', and then reset the index

df.drop(df.loc[df['Borough\n'] == 'Not assigned\n'].index, inplace=True)

df = df.reset_index(drop=True)
print('\n')
print('Size of dataframe after removing  --Borough-- that is --Not assigned--')
print('# Rows, Columns',df.shape)
print('\n')



Count of rows for each value of Boroughs  :
 Not assigned\n        77
North York\n          24
Downtown Toronto\n    19
Scarborough\n         17
Etobicoke\n           12
Central Toronto\n      9
West Toronto\n         6
East York\n            5
East Toronto\n         5
York\n                 5
Mississauga\n          1
Name: Borough\n, dtype: int64




Boroughs to drop due to --Not Assigned-- values :


    Postal Code\n       Borough\n Neighborhood\n
0           M1A\n  Not assigned\n             \n
1           M2A\n  Not assigned\n             \n
7           M8A\n  Not assigned\n             \n
10          M2B\n  Not assigned\n             \n
15          M7B\n  Not assigned\n             \n
..            ...             ...            ...
174         M4Z\n  Not assigned\n             \n
175         M5Z\n  Not assigned\n             \n
176         M6Z\n  Not assigned\n             \n
177         M7Z\n  Not assigned\n             \n
179         M9Z\n  Not assigned\n             \n

[77

In [13]:
# Remove the extraneous characters'\n' in every row, column 

df['Postal Code'] = df['Postal Code\n'].str.rstrip('\n')
df['Borough'] = df['Borough\n'].str.rstrip('\n')
df['Neighborhood'] = df['Neighborhood\n'].str.rstrip('\n')

df_new = df[['Postal Code','Borough','Neighborhood']]
df_new.head()
print('\n')
print ('Number of rows columns in the revised data frame', df_new.shape)
print('\n')



Number of rows columns in the revised data frame (103, 3)




In [14]:
# Check for rows where column Neighborhood has value has 'Not assigned'

df_Neighbor_NA = df_new.loc[df['Neighborhood'] == 'Not assigned']

#  Display the data frame that shows list of rows where 'Neighborhood' equals 'Not assigned'
print('\n')
print('Do any rows exist with no values for column Neighborhood ? ',df_Neighbor_NA)
print('\n')



Do any rows exist with no values for column Neighborhood ?  Empty DataFrame
Columns: [Postal Code, Borough, Neighborhood]
Index: []




In [15]:
# Final data frame size after scrapping data

print('\n')
print('Rows and Columns of the final data frame : ', df_new.shape)
print('\n')



Rows and Columns of the final data frame :  (103, 3)




In [ ]:
# Check the data to ensure the right records are displayed

In [16]:
df_new.iloc[0:9]

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"


In [17]:
df_new.iloc[100:105]

,Postal Code,Borough,Neighborhood
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."
102,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [ ]:
# Importing Longitude and Latitude for each postal codes

In [18]:
import pandas as pd
import numpy as np

In [19]:
filename = 'http://cocl.us/Geospatial_data'

In [21]:
# Read the contents of the file into a data frame, and assign column headers in the data fram

headers = ['Postal Code','Latitude','Longitude']

#df_pcode_long_lat = pd.read_csv(filename, names=headers)
df_pcode_long_lat = pd.read_csv(filename)

# Display the first 5 rows of the data frame

df_pcode_long_lat.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [22]:
# Check the columns of 2 dataframes to merge

# Dataframe with Postal code, Borough, Neighborr
print(df_new.columns)
print('\n')
print('Rows/Columns of PCode/Bor/Neigh :', df_new.shape)
print('\n')
# Dataframe with Postal code, Longitude, Latitude
print(df_pcode_long_lat.columns)
print('\n')
print('Rows/Columns of PCode/Lat/Lng :', df_pcode_long_lat.shape)

Index(['Postal Code', 'Borough', 'Neighborhood'], dtype='object')


Rows/Columns of PCode/Bor/Neigh : (103, 3)


Index(['Postal Code', 'Latitude', 'Longitude'], dtype='object')


Rows/Columns of PCode/Lat/Lng : (103, 3)


In [23]:
df_pcode_long_lat.tail()

,Postal Code,Latitude,Longitude
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437
102,M9W,43.706748,-79.594054


In [24]:
#Merge the 2 dataframes

merged_PCode = pd.merge(left=df_new, right=df_pcode_long_lat, left_on='Postal Code', right_on='Postal Code')

# Convert the Lat/Long to float from object
merged_PCode['Latitude'] = merged_PCode['Latitude'].astype(float)
merged_PCode['Longitude'] = merged_PCode['Longitude'].astype(float)

# What's the size of the output data?
merged_PCode.shape

(103, 5)

In [25]:
merged_PCode.dtypes

Postal Code      object
Borough          object
Neighborhood     object
Latitude        float64
Longitude       float64
dtype: object

In [26]:
# Display data frame with Postal Code, Borough, Neighborhood, Latitude, Longitude
merged_PCode

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [ ]:
# Segmentation and clustering

In [ ]:
# For this assignment, we pick up Boroughs containing the string 'Toronto' 
# After checking the count of neighborhoods for each borough,  ones with the name 'Toronto' had the largest number of rows.
# Second, on a couple of other neighborhoods, values for Latitudes and Longitudes were missing or NULL.

In [27]:
list_tor = merged_PCode['Borough'].str.contains('Toronto') 
Toronto_data = merged_PCode[list_tor]
Toronto_data = Toronto_data.reset_index(drop=True)

Toronto_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [28]:
Toronto_data['Neigh_split'] = Toronto_data['Neighborhood'].apply(lambda x: x.split(","))
Toronto_data.head()

Toronto_data['Neigh_count'] = Toronto_data['Neigh_split'].apply(len)
Toronto_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Neigh_split,Neigh_count
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,"[Regent Park, Harbourfront]",2
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,"[Queen's Park, Ontario Provincial Government]",2
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,"[Garden District, Ryerson]",2
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,[St. James Town],1
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,[The Beaches],1


In [29]:
# Define all the required libraries

import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0



geopy-1.22.0         | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: 

In [ ]:
# We pick the city of Toronto to find the lat, long

In [30]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="Tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Canada are 43.6534817, -79.3839347.


In [ ]:
# Create a map of Toronto, Canada with neighborhoods superimposed on top

In [31]:
# create map of Toronto using latitude and longitude values
map_tor = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Borough'], Toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    #print('Label :', label)
    label = folium.Popup(label, parse_html=True)
    #print('Label :', label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)
map_tor

In [32]:
# Define Four Square credentials

CLIENT_ID = 'JWBO22J423YEHZEJJLU0YJ3ZB4ZVVXRETYGWD4AGZ0PBMRPQ' # your Foursquare ID
CLIENT_SECRET = '5JL1QNJP0YAJJIPF0Y5KEVBQDBSZEDO1GEP4Q1O3EF4PZ10A' # your Foursquare SecreJWBO22J423YEHZEJJLU0YJ3ZB4ZVVXRETYGWD4AGZ0PBMRPQt
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JWBO22J423YEHZEJJLU0YJ3ZB4ZVVXRETYGWD4AGZ0PBMRPQ
CLIENT_SECRET:5JL1QNJP0YAJJIPF0Y5KEVBQDBSZEDO1GEP4Q1O3EF4PZ10A


In [ ]:
# Define an instance of the geocoder, define a user_agent. Name the agent 'foursquare_agent'

# Pick a Downtown location for Toronto to explore

#Toronto_data.loc[0, 'Neighborhood']

In [33]:
# Explore the neighborhoods of Toronto for the 'Subway'

# Pick a Downtown location for Toronto to explore
# Postal Code - M7A - 
# Borough - Downtown Toronto
# Neighborhood - Queen's Park, Ontario Provincial Government
# Latitude - 43.6623015
# Longitude - -79.3894938

latitude = 43.6623015
longitude = -79.3894938
print(latitude, longitude)

# Use Foursquare to explore venues around this location 
# Explore venues around 'Ontario Provincial Gov..' within 500 miles of this location

# Retrieve top 100 venues that are in neighborhood  of 'Queen's Park, Ontario Provincial Government'
# within a radius of 500 meters

search_query = 'Ontario Provincial Government'
radius = 500
LIMIT = 100 # Limit the number 
print(search_query + ' .... OK!')

# Define the url to explore

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

43.6623015 -79.3894938
Ontario Provincial Government .... OK!


'https://api.foursquare.com/v2/venues/explore?client_id=JWBO22J423YEHZEJJLU0YJ3ZB4ZVVXRETYGWD4AGZ0PBMRPQ&client_secret=5JL1QNJP0YAJJIPF0Y5KEVBQDBSZEDO1GEP4Q1O3EF4PZ10A&ll=43.6623015,-79.3894938&v=20180604&query=Ontario Provincial Government&radius=500&limit=100'

In [34]:
# Send the request and examine the results

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ed196a495feaf001bd89226'},
 'response': {'warning': {'text': 'There aren\'t a lot of results for "Ontario Provincial Government." Try something more general, reset your filters, or expand the search area.'},
  'headerLocation': 'Bay Street Corridor',
  'headerFullLocation': 'Bay Street Corridor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'query': 'ontario provincial government',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 43.6668015045, 'lng': -79.38328496866619},
   'sw': {'lat': 43.657801495499996, 'lng': -79.3957026313338}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bc3afb2461576b0bd077f32',
       'name': 'Ontario Fire Fighters Memorial',
       'location': {'crossStreet': "University Ave & Que

In [35]:
#get_category_type function from the Foursquare

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [36]:
# Information is in the items key

# Clean the json and structure it into a pandas dataframe

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  import sys


,name,categories,lat,lng
0,Ontario Fire Fighters Memorial,Sculpture Garden,43.660335,-79.389921


In [37]:
print('{} venues were returned by Foursquare for neighborhood of Ontario Prov Gov.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare for neighborhood of Ontario Prov Gov.


In [39]:
# Explore neighborhoods Downtown Toronto

# Create a function to check all the neighborhoods

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [40]:
Toronto_venues = getNearbyVenues(names=Toronto_data['Neighborhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport


In [41]:
print(Toronto_venues.shape)
Toronto_venues.head()

(1621, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [42]:
# How many venues were returned for each neighborhood
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,58,58,58,58,58,58
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",17,17,17,17,17,17
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",14,14,14,14,14,14
Central Bay Street,63,63,63,63,63,63
Christie,17,17,17,17,17,17
Church and Wellesley,78,78,78,78,78,78
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,34,34,34,34,34,34


In [46]:
# Analyze Each Neighborhood

# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
Toronto_onehot.shape

(1621, 237)

In [48]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.071429,0.071429,0.142857,0.142857,0.142857,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.015873,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.015873,0.000000,0.000000,0.015873,0.000000,0.000000
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.025641,0.012821,0.000000,0.000000,0.000000,0.000000,0.000000,0.012821,0.000000,...,0.012821,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.012821,0.000000
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.029412,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [49]:
#Let's print each neighborhood along with the top 5 most common venues

num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
          venue  freq
0   Coffee Shop  0.07
1  Cocktail Bar  0.05
2      Beer Bar  0.03
3           Pub  0.03
4          Café  0.03


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.13
1       Nightclub  0.09
2  Breakfast Spot  0.09
3     Coffee Shop  0.09
4      Restaurant  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue  freq
0    Light Rail Station  0.12
1           Yoga Studio  0.06
2            Restaurant  0.06
3            Smoke Shop  0.06
4  Fast Food Restaurant  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0    Airport Lounge  0.14
1   Airport Service  0.14
2  Airport Terminal  0.14
3             Plane  0.07
4          Boutique  0.07


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.17
1  Italian Resta

In [50]:
# function to sort the venues in descending order

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [51]:
# Create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Pub,Cheese Shop,Restaurant,Seafood Restaurant,Bakery,Beer Bar,Café,Shopping Mall
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Nightclub,Grocery Store,Stadium,Burrito Place,Restaurant,Climbing Gym,Pet Store
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Auto Workshop,Smoke Shop,Brewery,Spa,Farmers Market,Fast Food Restaurant,Burrito Place,Butcher,Restaurant
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Boutique,Airport,Airport Food Court,Harbor / Marina,Bar,Sculpture Garden
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Burger Joint,Ice Cream Shop,Japanese Restaurant,Salad Place,Bubble Tea Shop,Yoga Studio


In [52]:
neighborhoods_venues_sorted.shape

(39, 11)

In [53]:
#  Run k-means to cluster the neighborhood into 5 clusters.

# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [54]:
# Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = Toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Neigh_split,Neigh_count,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,"[Regent Park, Harbourfront]",2,0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Theater,Mexican Restaurant,Shoe Store,Restaurant
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,"[Queen's Park, Ontario Provincial Government]",2,0,Coffee Shop,Yoga Studio,Creperie,Beer Bar,Smoothie Shop,Sandwich Place,Burger Joint,Burrito Place,Restaurant,Café
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,"[Garden District, Ryerson]",2,0,Clothing Store,Coffee Shop,Café,Middle Eastern Restaurant,Cosmetics Shop,Italian Restaurant,Japanese Restaurant,Restaurant,Bubble Tea Shop,Tea Room
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,[St. James Town],1,0,Café,Coffee Shop,Cocktail Bar,American Restaurant,Gastropub,Restaurant,Hotel,Cosmetics Shop,Creperie,Lingerie Store
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,[The Beaches],1,0,Park,Coffee Shop,Trail,Health Food Store,Pub,Women's Store,Distribution Center,Department Store,Dessert Shop,Diner


In [55]:
# Visualize  neighborhood on each cluster

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters